In [1]:
import faiss
import sqlite3
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

# Load model

/Users/cam/Documents/diss/copy/synoptic-project-CameronFMacKay/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)

In [ ]:
df = pd.read_csv('combined.csv')
df = df.rename(columns={'Lyric': 'lyrics', 'Title': 'title', 'Artist': 'artist'})
df = df.drop(columns={'Unnamed: 0', 'Album', 'Year', 'Date'})

In [57]:
df.head()

,artist,title,lyrics
0,Dua Lipa,New Rules,one one one one one talkin' in my sleep at n...
1,Dua Lipa,Don’t Start Now,if you don't wanna see me did a full 80 craz...
2,Dua Lipa,IDGAF,you call me all friendly tellin' me how much y...
3,Dua Lipa,Blow Your Mind (Mwah),i know it's hot i know we've got something tha...
4,Dua Lipa,Be the One,i see the moon i see the moon i see the moon o...


In [41]:
march = pd.read_csv('march25hot.csv')
march = march.drop(columns={"song_id"})
march.head()

,title,artist,lyrics
0,Kendrick Lamar & SZA - luther (Tradução em Por...,Genius Brasil Traduções,[Intro]\nSe esse mundo fosse meu\n\n[Verso 1: ...
1,Die With A Smile,Lady Gaga,"[Intro: Bruno Mars]\n(Ooh, ooh)\n\n[Verse 1: B..."
2,Not Like Us,Kendrick Lamar,"[Intro]\nPsst, I see dead people\n(Mustard on ..."
3,A Bar Song (Tipsy),Shaboozey,"[Verse 1]\nMy baby want a Birkin, she's been t..."
4,Kendrick Lamar - tv off ft. Lefty Gunplay (Tür...,Genius Türkçe Çeviri,"[""tv off"" için Türkçe şarkı sözleri]\n\n[Kısım..."


In [58]:
test = df


In [43]:
df.head()

,artist,title,lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [59]:
embeddings = model.encode(test['lyrics'].tolist(), batch_size=64, convert_to_numpy=True, device=device)

TypeError: 'float' object is not subscriptable

In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 Distance Index
index.add(embeddings)
faiss.write_index(index, "song_lyrics3.index")


In [ ]:
db_conn = sqlite3.connect("songs3.db")
df.to_sql("songs", db_conn, if_exists="replace", index=True)
db_conn.close()

In [50]:
df.count()

artist    57650
title     57650
lyrics    57650
dtype: int64

In [52]:
db_conn = sqlite3.connect("songs2.db")
cursor = db_conn.cursor()
cursor.execute("SELECT * FROM songs WHERE rowid = ?", (57630,))
row = cursor.fetchall()
print(row)


[(57629, 'Zebrahead', 'House Is Not My Home', "Well I'm tired, so tired of getting up and wasting another day  \r\nI wish the picture on the wall would still say it all  \r\nBut now yesterday is so far away  \r\nAnd I would give anything, say anything  \r\n  \r\nTo keep it all like it was  \r\nBut you just throw it all away  \r\nAnd I think this house is not my home  \r\nI think that I'd rather be alone tonight  \r\nI'm trying, trying to fake it  \r\n  \r\nI'm tired, so tired, of giving up and taking all the blame  \r\nI wish the shouting and the fights would stop for the night  \r\nCan't close my eyes and make them all go away  \r\nAnd I would give anything, say anything  \r\nTo keep it all like it was  \r\nBut you just throw it all away  \r\n  \r\n[Chorus]  \r\n  \r\nThis is the part where you say  \r\nYou can't stay together just for me  \r\nThis is the part where you make me choose  \r\nOn whose side I want to be  \r\nThis is the part where you decide  \r\nWhat's going to be best f

In [33]:
def search_songs(tag, top_k=5):
    tag_embedding = model.encode([tag], convert_to_numpy=True, device=device)
    index = faiss.read_index("song_lyrics.index")
    distances, indices = index.search(tag_embedding, top_k)
    # Retrieve song metadata
    db_conn = sqlite3.connect("songs.db")
    cursor = db_conn.cursor()
    
    results = []
    for idx, distance in zip(indices[0], distances[0]):
        cursor.execute("SELECT artist, title, lyrics FROM songs WHERE rowid = ?", (int(idx)+1,))
        row = cursor.fetchone()
        print(row)
        if row:
            artist, title, lyrics = row
            results.append((artist, title, lyrics, distance))
    
    db_conn.close()
    return results

In [53]:
tag = "they not like us"
results = search_songs(tag)
if results:
    print("Most similar song:")
    print(f"Title: {results[1]}\nLyrics: {results[3][:200]}...\nSimilarity Score: {results[4]:.4f}\n")
else:
    print("No matching songs found.")

('MF DOOM', 'Deep Fried Frenz', 'intro\nbefore we go any further\nfriends\n\nverse\nas you call em they call you when they need something\ntrees for the bluntin to gs for the frontin\ni found a way to get peace of mind for years and left the hell alone\nturn a deaf ear to the cellular phone\nsend me a letter or better we could see each other in real life\njust so you could feel me like a steel knife\nat least so you could see the white of they eyes\nbright with surprise once they finish spitting lies\nassociates is your boys your girls bitches niggas homies\nclose or really dont know me\nmom dad comrade peeps brothers sisters duns dunnies\nsome come around when they need some money\nothers make us laugh like the sunday funnies\nfam be around whether you paid or bummy\nyou could either ignore this advice or take it from me\nbe too nice and people take you for a dummy\nso nowadays he aint so friendly\nactually they wouldn’t even made a worthy enemy\nread the signs no feeding the baboons\

TypeError: unsupported format string passed to tuple.__format__